# Ragas Evaluation Demo  -- Remote execution on Kubeflow Pipelines

- This notebook demonstrates how to run Ragas evaluation on Kubeflow Pipelines.
- See README.md for details on how to setup the demo.


## Imports


In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompt_values import StringPromptValue
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from ragas.llms.base import LangchainLLMWrapper
from rich.pretty import pprint

from magenta_pipelines.config import EvalConfig, KubeflowConfig
from magenta_pipelines.pipeline_runner import PipelineRunner

load_dotenv()

/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Load configs from .env

In [2]:
# load configs from .env
kfp_config = KubeflowConfig()

## Inference setup (generations and embeddings)
- You can use any OpenAI API compatible model and wrap it with the Ragas LangchainLLMWrapper or LangchainEmbeddingsWrapper
- This example uses Ollama

In [3]:
llm_client = LangchainLLMWrapper(
    OllamaLLM(model="granite3.3:2b", base_url=os.environ["INFERENCE_URL"])
)
llm_client.generate_text(
    StringPromptValue(text="Tell me a joke")
)  # StringPromptValue is the way Ragas will invoke the LLM

LLMResult(generations=[[GenerationChunk(text="Why don't scientists trust atoms? Because they make up everything!", generation_info={'model': 'granite3.3:2b', 'created_at': '2025-08-26T01:46:29.487983Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1398585709, 'load_duration': 1103697375, 'prompt_eval_count': 48, 'prompt_eval_duration': 103471458, 'eval_count': 16, 'eval_duration': 190822500, 'response': '', 'thinking': None, 'context': [49152, 2946, 49153, 39558, 390, 17071, 2821, 44, 30468, 225, 36, 34, 36, 38, 32, 203, 4282, 884, 8080, 278, 659, 30, 18909, 810, 25697, 32, 2448, 884, 312, 17247, 19551, 47330, 32, 0, 203, 49152, 496, 49153, 46020, 597, 312, 8725, 479, 0, 203, 49152, 17594, 49153, 24402, 2800, 1330, 2197, 1606, 3045, 18074, 27898, 49, 18369, 2953, 1930, 973, 8989, 19]})]], llm_output=None, run=[RunInfo(run_id=UUID('9e2a7bb2-d3b9-4acb-80ac-968aecee61ce'))], type='LLMResult')

In [4]:
embedding_client = LangchainEmbeddingsWrapper(
    OllamaEmbeddings(
        model="all-minilm:l6-v2",
        base_url=os.environ["INFERENCE_URL"],
    )
)
embedded_documents = embedding_client.embed_documents(["Hello, world!"])
assert len(embedded_documents[0]) == 384  # l6-v2 has 384 dimensions

## Dataset Preparation

Create a sample RAG evaluation dataset. In a real scenario, you would load your own dataset.


In [5]:
# Sample Ragas evaluation dataset
evaluation_data = [
    {
        "user_input": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "retrieved_contexts": [
            "Paris is the capital and most populous city of France."
        ],
        "reference": "Paris",
    },
    {
        "user_input": "Who invented the telephone?",
        "response": "Alexander Graham Bell invented the telephone in 1876.",
        "retrieved_contexts": [
            "Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone."
        ],
        "reference": "Alexander Graham Bell",
    },
    {
        "user_input": "What is photosynthesis?",
        "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
        "retrieved_contexts": [
            "Photosynthesis is a process used by plants to convert light energy into chemical energy."
        ],
        "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
    },
]

## Upload the dataset to S3

Register the dataset with Llama Stack's Datasets API using the direct rows approach.


In [6]:
S3_PREFIX = "s3://public-rhods/ragas-evaluation-pipeline"
dataset_id = "ragas_demo_dataset_remote"
input_dataset_uri = f"{S3_PREFIX}/{dataset_id}_input.jsonl"
output_dataset_uri = f"{S3_PREFIX}/{dataset_id}_output.jsonl"

In [7]:
# upload the dataset to S3
pd.DataFrame(evaluation_data).to_json(input_dataset_uri, orient="records", lines=True)

## Evaluation Execution

In [8]:
pipeline_runner = PipelineRunner(kfp_config)


/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.13/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [9]:
eval_config = EvalConfig(
    model="granite3.3:2b",
    model_params={"temperature": 0.1, "max_tokens": 100},
    embedding_model="all-minilm:l6-v2",
    metric_names=["answer_relevance"],
    input_dataset_uri=input_dataset_uri,
    output_dataset_uri=output_dataset_uri,
    # inference_url is read from .env
)
job = pipeline_runner.run_eval(eval_config)
pprint(job)

EvalJob(
│   job_id='6b7adf44-f5f1-49a0-9330-d05d21b50f72',
│   status='submitted',
│   eval_config=EvalConfig(
│   │   input_dataset_uri='s3://public-rhods/ragas-evaluation-pipeline/ragas_demo_dataset_remote_input.jsonl',
│   │   output_dataset_uri='s3://public-rhods/ragas-evaluation-pipeline/ragas_demo_dataset_remote_output.jsonl',
│   │   inference_url='https://83e9089ea2c2.ngrok-free.app',
│   │   model='granite3.3:2b',
│   │   model_params={'temperature': 0.1, 'max_tokens': 100},
│   │   embedding_model='all-minilm:l6-v2',
│   │   metric_names=['answer_relevancy', 'context_precision', 'faithfulness', 'context_recall'],
│   │   ragas_config=RagasConfig(batch_size=None, show_progress=True, raise_exceptions=True, column_map=None)
│   ),
│   kubeflow_run_id='c82567ad-644c-42ce-b91f-22d06378ce29',
│   result=None
)

## 8. Results Display


In [10]:
pprint(pipeline_runner.job_status(job_id=job.job_id))

EvalJob(
│   job_id='6b7adf44-f5f1-49a0-9330-d05d21b50f72',
│   status='in_progress',
│   eval_config=EvalConfig(
│   │   input_dataset_uri='s3://public-rhods/ragas-evaluation-pipeline/ragas_demo_dataset_remote_input.jsonl',
│   │   output_dataset_uri='s3://public-rhods/ragas-evaluation-pipeline/ragas_demo_dataset_remote_output.jsonl',
│   │   inference_url='https://83e9089ea2c2.ngrok-free.app',
│   │   model='granite3.3:2b',
│   │   model_params={'temperature': 0.1, 'max_tokens': 100},
│   │   embedding_model='all-minilm:l6-v2',
│   │   metric_names=['answer_relevancy', 'context_precision', 'faithfulness', 'context_recall'],
│   │   ragas_config=RagasConfig(batch_size=None, show_progress=True, raise_exceptions=True, column_map=None)
│   ),
│   kubeflow_run_id='c82567ad-644c-42ce-b91f-22d06378ce29',
│   result=None
)

In [14]:
# wait a few moments for the pipeline to finish
pprint(pipeline_runner.job_status(job_id=job.job_id))

                                               Fetched Evaluation Results                                               
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ user_input   ┃ retrieved_c… ┃ response     ┃ reference    ┃ answer_rele… ┃ context_pre… ┃ faithfulness ┃ context_re… ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ What is the  │ ['Paris is   │ The capital  │ Paris        │ 0.962092759  │ 0.9999999999 │ 1            │ 0.66666666… │
│ capital of   │ the capital  │ of France is │              │              │              │              │             │
│ France?      │ and most     │ Paris.       │              │              │              │              │             │
│              │ populous     │              │              │              │              │              │             │
│              │ city of      │              │              │              │              │              │             │
│              │ France.']    │              │              │              │              │              │             │
│ Who invented │ ['Alexander  │ Alexander    │ Alexander    │ 1.0          │ 0.9999999999 │ 1            │ 0.66666666… │
│ the          │ Graham Bell  │ Graham Bell  │ Graham Bell  │              │              │              │             │
│ telephone?   │ was a        │ invented the │              │              │              │              │             │
│              │ Scottish-Am… │ telephone in │              │              │              │              │             │
│              │ inventor who │ 1876.        │              │              │              │              │             │
│              │ patented the │              │              │              │              │              │             │
│              │ first        │              │              │              │              │              │             │
│              │ practical    │              │              │              │              │              │             │
│              │ telephone.'] │              │              │              │              │              │             │
│ What is      │ ['Photosynt… │ Photosynthe… │ Photosynthe… │ 1.0          │ 0.9999999999 │ 1            │ 0.66666666… │
│ photosynthe… │ is a process │ is the       │ is the       │              │              │              │             │
│              │ used by      │ process by   │ process by   │              │              │              │             │
│              │ plants to    │ which plants │ which plants │              │              │              │             │
│              │ convert      │ convert      │ and other    │              │              │              │             │
│              │ light energy │ sunlight     │ organisms    │              │              │              │             │
│              │ into         │ into energy. │ convert      │              │              │              │             │
│              │ chemical     │              │ light energy │              │              │              │             │
│              │ energy.']    │              │ into         │              │              │              │             │
│              │              │              │ chemical     │              │              │              │             │
│              │              │              │ energy.      │              │              │              │             │
└──────────────┴──────────────┴──────────────┴──────────────┴──────────────┴──────────────┴──────────────┴─────────────┘

EvalJob(
│   job_id='6b7adf44-f5f1-49a0-9330-d05d21b50f72',
│   status='completed',
│   eval_config=EvalConfig(
│   │   input_dataset_uri='s3://public-rhods/ragas-evaluation-pipeline/ragas_demo_dataset_remote_input.jsonl',
│   │   output_dataset_uri='s3://public-rhods/ragas-evaluation-pipeline/ragas_demo_dataset_remote_output.jsonl',
│   │   inference_url='https://83e9089ea2c2.ngrok-free.app',
│   │   model='granite3.3:2b',
│   │   model_params={'temperature': 0.1, 'max_tokens': 100},
│   │   embedding_model='all-minilm:l6-v2',
│   │   metric_names=['answer_relevancy', 'context_precision', 'faithfulness', 'context_recall'],
│   │   ragas_config=RagasConfig(batch_size=None, show_progress=True, raise_exceptions=True, column_map=None)
│   ),
│   kubeflow_run_id='c82567ad-644c-42ce-b91f-22d06378ce29',
│   result=                       user_input  \
0  What is the capital of France?   
1     Who invented the telephone?   
2         What is photosynthesis?   
│   │   │   │   │   │   │   │     
│   │   │   │   │   │   │   │     retrieved_contexts  \
0  [Paris is the capital and most populous city o...   
1  [Alexander Graham Bell was a Scottish-American...   
2  [Photosynthesis is a process used by plants to...   
│   │   │   │   │   │   │   │   │   │   │   
│   │   │   │   │   │   │   │   │   │   │   response  \
0                    The capital of France is Paris.   
1  Alexander Graham Bell invented the telephone i...   
2  Photosynthesis is the process by which plants ...   
│   │   │   │   │   │   │   │   │   │      
│   │   │   │   │   │   │   │   │   │      reference  answer_relevancy  \
0                                              Paris          0.962093   
1                              Alexander Graham Bell          1.000000   
2  Photosynthesis is the process by which plants ...          1.000000   
   
   context_precision  faithfulness  context_recall  
0                1.0             1        0.666667  
1                1.0             1        0.666667  
2                1.0             1        0.666667  
)

In [15]:
pprint(pipeline_runner.job_result(job_id=job.job_id))

│   │   │   │   │      user_input  \
0  What is the capital of France?   
1     Who invented the telephone?   
2         What is photosynthesis?   
│   │   │   │   │   │   │   │     
│   │   │   │   │   │   │   │     retrieved_contexts  \
0  [Paris is the capital and most populous city o...   
1  [Alexander Graham Bell was a Scottish-American...   
2  [Photosynthesis is a process used by plants to...   
│   │   │   │   │   │   │   │   │   │   │   
│   │   │   │   │   │   │   │   │   │   │   response  \
0                    The capital of France is Paris.   
1  Alexander Graham Bell invented the telephone i...   
2  Photosynthesis is the process by which plants ...   
│   │   │   │   │   │   │   │   │   │      
│   │   │   │   │   │   │   │   │   │      reference  answer_relevancy  \
0                                              Paris          0.962093   
1                              Alexander Graham Bell          1.000000   
2  Photosynthesis is the process by which plants ...          1.000000   
   
   context_precision  faithfulness  context_recall  
0                1.0             1        0.666667  
1                1.0             1        0.666667  
2                1.0             1        0.666667